<a href="https://colab.research.google.com/github/cmannnn/california_housing/blob/main/california_housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Housing Corp. - California Housing

The first task you are asked to perform is to build a model of housing prices in California using the California cen‐ sus data. This data has metrics such as the population, median income, median hous‐ ing price, and so on for each block group in California. Block groups are the smallest geographical unit for which the US Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). We will just call them “dis‐ tricts” for short.

**Your task is to predict median house values in Californian districts, given a number of features from these districts.**

#Imports

In [18]:
import sys
print(f'Using sys version: {sys.version_info}')

import matplotlib.pyplot as plt
import matplotlib
print(f'Using matplotlib version: {matplotlib.__version__}')

import numpy as np
print(f'Using numpy version: {np.__version__}')

import pandas as pd
print(f'Using pandas version: {pd.__version__}')